In [ ]:
import seaborn as sns
import pandas as pd
%pylab inline

In [ ]:
df_lalonde = pd.read_csv(r'lalonde.csv')
df_lalonde.head()

In [ ]:
df_lalonde.describe()

In [ ]:
df_treat = df_lalonde[df_lalonde['treat']==1]
df_notreat = df_lalonde[df_lalonde['treat']==0]

fig, axes = plt.subplots(1, 2, sharey=True)
df_treat.re78.plot(kind='hist', ax=axes[0])
df_notreat.re78.plot(kind='hist', ax=axes[1])
print(df_lalonde[df_lalonde['treat']==1].re78.describe())
df_notreat.re78.describe()

We can see some interesting trends if we start correlating the different characteristics within the groups.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
# todo: choose better color scheme

sns.heatmap(df_treat.corr(method='spearman').values, ax=axes[0], center=0, 
            xticklabels=df_treat.columns[1:], yticklabels=df_lalonde.columns[1:])
sns.heatmap(df_notreat.corr(method='spearman').values, ax=axes[1], center=0, 
            xticklabels=df_notreat.columns[1:], yticklabels=df_lalonde.columns[1:])

In [ ]:
sns.heatmap(df_lalonde.corr().values, center=0, 
            xticklabels=df_lalonde.columns[1:], yticklabels=df_lalonde.columns[1:])